In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#### 1.设置输入和输出节点的个数,配置神经网络的参数。

In [4]:
INPUT_NODE = 784     # 输入节点。图片像素28*28
OUTPUT_NODE = 10     # 输出节点。类别数目
LAYER1_NODE = 500    # 隐藏层节点数       
                              
BATCH_SIZE = 100     # 每次batch打包的样本个数        

# 模型相关的参数
LEARNING_RATE_BASE = 0.8     # 基础学习率 
LEARNING_RATE_DECAY = 0.99   # 学习率的衰减率
REGULARAZTION_RATE = 0.0001  # 描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS = 5000        # 训练轮数
MOVING_AVERAGE_DECAY = 0.99  # 滑动平均衰减率

#### 2. 定义辅助函数来计算前向传播结果，使用ReLU做为激活函数。
给定神经网络的输入和所有参赛，计算神经网络的前向传播结果。通过ReLU激活函数实现去线性化。

因为在计算损失函数时会一并计算softmax函数，所以，在计算输出层的前向传播结果时，不需要加入激活函数。而且不会影响预测结果，因为预测时使用的是不同类别对应节点输出值的相对大小，有没有softmax层对最后的分类结果的计算没有影响。于是，在计算整个神经网络的前向传播时可以不加入最后的softmax层。

In [5]:
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 不使用滑动平均类
    if avg_class == None:
        # 计算隐藏层的前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        # 计算输出层的前向传播结果
        return tf.matmul(layer1, weights2) + biases2

    else:
        # 使用avg_class.average计算出变量的滑动平均值，然后再计算相应的神经网络前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)  

#### 3. 定义训练过程。
**计算交叉熵**

`tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_))`

* y：神经网络不包含softmax层的前向传播结果
* y_：训练数据的正确答案
* tf.argmax(y_)：在这里，因为正确答案是一个长度为10的一维数组，而该函数需要提供的是一个正确答案的数字，所以需要tf.argmax函数来得到正确答案对应的类别编号

函数是将softmax和cross_entropy放在一起计算，对于分类问题而言，最后一般都是一个单层全连接神经网络，比如softmax分类器居多。

对该函数而言，tensorflow神经网络中是没有softmax层，而是在这个函数中进行softmax函数的计算。该函数是直接使用标签数据，而不是采用one-hot编码形式。

当分类问题只有一个正确答案时，可以使用这个函数来加速交叉熵的计算。

**tf.argmax(average_y, 1)**

计算每一个样例的预测答案。average_y是一个batch_size*10的二维数组，每一行表示一个样例的前向传播结果。tf.argmax的第二个参数“1”表示选取最大值的操作仅在第一个维度中进行，即只在每一行选取最大值对应的下标。

所以该运算得到的结果是一个长度为batch的一维数组，值表示每个样例的数字识别结果。

tf.equal判断两个张量的每一维是否相等，相等返回true，否则返回false。

In [6]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量。因为不需要计算滑动平均值，所以指定为不可训练的变量（trainable=False）
    global_step = tf.Variable(0, trainable=False)
    
    # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    # 在所有打标神经网络参数的变量上使用滑动平均
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    # 计算使用滑动平均之后的前向传播结果
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    # 计算模型的正则化损失。一般只计算神经网络边上权重的正则化损失，而不使用偏置项
    regularaztion = regularizer(weights1) + regularizer(weights2)
    # 总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularaztion
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, # 过完所有训练数据需要迭代的次数
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 在训练神经网络模型时，每过一遍数据即需要通过反向传播来更新神经网络中的参数，
    # 又需要更新每一个参数的滑动平均值。
    # tf中提供了两种方法来一次完成多个操作：tf.control_dependencies 和 tf.group
    # with tf.control_dependencies([train_step, variables_averages_op]):
    #     train_op = tf.no_op(name='train')
    train_op = tf.group(train_step, variables_averages_op)    

    #检验使用滑动平均模型的神经网络前向传播是否正确
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 将布尔型的数值转换为实数型，然后计算平均值。该平均值即为模型在一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话，并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 准备验证数据
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        # 准备测试数据
        # 注意：在真实应用中，这部分数据在训练时是不可见的，该数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证集上的测试结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x:xs,y_:ys})
        
        # 训练结束后，在测试数据上检测神经网络模型的最终正确率
        test_acc=sess.run(accuracy,feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" %(TRAINING_STEPS, test_acc)))
   

#### 4. 主程序入口，这里设定模型训练次数为5000次。

In [8]:
def main(argv=None):
    import logging
    logging.basicConfig(level=logging.ERROR)
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__=='__main__':
    main()

Extracting ../../datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.133 
After 1000 training step(s), validation accuracy using average model is 0.9784 
After 2000 training step(s), validation accuracy using average model is 0.9812 
After 3000 training step(s), validation accuracy using average model is 0.9822 
After 4000 training step(s), validation accuracy using average model is 0.9846 
After 5000 training step(s), test accuracy using average model is 0.9832
